In [1]:
import os
import torch
import pandas as pd
from modules.feature_engineering import FeatureEngineeringV1 
from modules.data_module import FlightDataModule
from modules.training import TrainingFlightModel
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger

print(torch.__version__)
fe = FeatureEngineeringV1()

2.1.2+cu121


In [2]:
%load_ext tensorboard

In [3]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        print(f'Using device: {torch.cuda.get_device_name(0)}')
        return torch.device('cuda')
    elif torch.backends.mps.is_available():
        print('Using device: Apple ARM GPU')
        return torch.device('mps')
    else:
        print('Using device: CPU')
        return torch.device('cpu')
    
# Set the device
device = get_default_device()
print(f'Using device: {device}')

Using device: NVIDIA GeForce RTX 3060
Using device: cuda


In [4]:
# Used with GPUs having Tensor cores
torch.set_float32_matmul_precision('medium')

In [5]:
processed_df = pd.read_csv('../data/csv/processed.csv')
processed_df

,wind_speed_TSN,wind_dir_TSN,wind_speed_KADUM,wind_dir_KADUM,wind_speed_PATNA,wind_dir_PATNA,wind_speed_SADAS,wind_dir_SADAS,wind_speed_DADEN,wind_dir_DADEN,...,Ac_kts,Time_step,Ac_Lat,Ac_Lon,Ac_feet,Ac_type_1,Ac_type_2,Phase_1,Phase_2,Phase_3
0,0.928702,0.495425,-0.634564,0.022136,-1.145864,2.972182,-2.250708,5.614973,-1.593272,2.974218,...,-2.268434,-1.526071,-1.407072,-0.227452,-1.853900,1.0,0.0,1.0,0.0,0.0
1,0.928702,0.495425,-0.634564,0.023238,-1.145864,2.980370,-2.250708,5.623375,-1.593272,2.963249,...,-2.183981,-1.515271,-1.408670,-0.245917,-1.791555,1.0,0.0,1.0,0.0,0.0
2,0.928702,0.495425,-0.634564,0.023238,-1.145864,2.980370,-2.250708,5.623375,-1.593272,2.963249,...,-1.940006,-1.504471,-1.409917,-0.266631,-1.762331,1.0,0.0,1.0,0.0,0.0
3,0.928702,0.495425,-0.634564,0.024339,-1.145864,2.988558,-2.315868,5.640178,-1.593272,2.957764,...,-1.696031,-1.495831,-1.408996,-0.284978,-1.744796,1.0,0.0,1.0,0.0,0.0
4,0.928702,0.495425,-0.634564,0.024339,-1.145864,2.988558,-2.315868,5.640178,-1.593272,2.957764,...,-1.470824,-1.487191,-1.405175,-0.300484,-1.709727,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,0.0
61381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,0.0
61382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,0.0
61383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,0.0


In [20]:
processed_df['Ac_id'].unique()

array(['R6-VJ150-21', 'R6-BL6018-21', 'R6-VJ132-22', 'R6-QH9216-22',
       'R6-QH262-22', 'R6-BL6046-22', 'R6-VN248-23', 'R6-VJ198-23',
       'R6-QH246-23', 'R6-VJ162-23', 'R6-VJ150-24', 'R6-BL6018-24',
       'R6-VN248-24', 'R6-VJ198-24', 'R6-VN7222-25', 'R6-BL6048-25',
       'R6-VJ126-25', 'R6-QH224-25', 'R6-QH204-26', 'R6-VJ122-26',
       'R6-VU750-26', 'R6-VN244-26', 'R6-VJ150-27', 'R6-VJ6018-27',
       'R6-VN248-27', 'R6-VJ198-27', 'R6-VN208-04', 'R6-VN7222-04',
       'R6-BL6046-04', 'R6-VJ122-03', 'R6-BL6002-03', 'R6-VN208-03',
       'R6-QH204-03', 'R6-VJ156-02', 'R6-VN244-02', 'R6-BL6002-02',
       'R6-VJ122-02', 'R6-QH204-02', 'R6-BL6046-01', 'R6-BL6048-01',
       'R6-VN244-01', 'R6-VN208-01', 'R6-VJ122-01', 'R6-BL6046-31',
       'R6-VJ126-31', 'R6-BL6048-31', 'R6-VN244-31', 'R6-VN7222-31',
       'R6-VJ122-31', 'R6-VJ132-30', 'R6-VN216-30', 'R6-VJ162-30',
       'R6-VN246-30', 'R6-VN248-30', 'R6-BL6046-29', 'R6-VJ6126-29',
       'R6-QH262-29', 'R6-VN258-29', 'R6-VJ1

In [21]:
X, y = fe.padding_features(processed_df)
X[352]

array([], shape=(0, 42), dtype=float64)

In [7]:
X_train, X_val, X_test, y_train, y_val, y_test = fe.split_train_test(X, y, test_size=0.2, val_size=0.25)
print(f'Shape of training set is {X_train.shape}, {y_train.shape}')
print(f'Shape of validation set is {X_val.shape}, {y_val.shape}')
print(f'Shape of test set is {X_test.shape}, {y_test.shape}')

Shape of training set is (281, 6616, 36), (281, 6616, 4)
Shape of validation set is (53, 6616, 36), (53, 6616, 4)
Shape of test set is (18, 6616, 36), (18, 6616, 4)


In [8]:
train_dataset, val_dataset, test_dataset = fe.make_tensor_dataset(X_train, X_val, X_test, y_train, y_val, y_test)
print(f'training set is {train_dataset}')
print(f'validation set is {val_dataset}')
print(f'test set is {test_dataset}')

training set is <torch.utils.data.dataset.TensorDataset object at 0x7fce24034c50>
validation set is <torch.utils.data.dataset.TensorDataset object at 0x7fce23f9ec90>
test set is <torch.utils.data.dataset.TensorDataset object at 0x7fce240344d0>


# TRAINING

---

In [9]:
# Global variables
batch_size_variable = 16
max_epochs = 200

In [10]:
flight_dm = FlightDataModule(train_dataset, val_dataset, test_dataset, batch_size=batch_size_variable)

In [11]:
model_name = 'flight_model_LSTM_V1'
base_dir = os.getcwd().replace('notebook', '')
log_dir = os.path.join(base_dir, 'log')
ckpt_dir = os.path.join(base_dir, model_name)
os.makedirs(log_dir, exist_ok=True)

In [12]:
log_dir

'/mnt/e/Work/chi_phuong_co_su/log'

In [13]:
# Some necessary callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=True, mode='min')
model_checkpoint = ModelCheckpoint(
    dirpath=log_dir,
    filename='LSTM-{epoch}-{val_loss:.5f}',
    monitor='val_loss', 
    save_top_k=3, 
    verbose=True, 
    mode='min'
)
logger = TensorBoardLogger(log_dir, name=model_name, log_graph=True)

In [14]:
training_module = TrainingFlightModel(
    early_stopping_lstm = early_stopping,
    model_checkpoint_lstm = model_checkpoint,
    logger_lstm = logger,
    data_module = flight_dm,
    max_epoch=max_epochs,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [15]:
training_module.training(
    input_size=36, 
    hidden_size=300, 
    num_layers=10,
    output_size=4, 
    learning_rate=1e-4,
    dropout=0.1,
)

Missing logger folder: /mnt/e/Work/chi_phuong_co_su/log/flight_model_LSTM_V0
/home/anvutrong/miniconda3/envs/wsl_env/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:653: Checkpoint directory /mnt/e/Work/chi_phuong_co_su/log exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type             | Params | In sizes                                    | Out sizes                                   
---------------------------------------------------------------------------------------------------------------------------------------
0 | lstm | LSTM             | 3.3 M  | [[1, 6616, 36], [[5, 1, 300], [5, 1, 300]]] | [[1, 6616, 300], [[5, 1, 300], [5, 1, 300]]]
1 | fc   | Linear           | 1.2 K  | [1, 6616, 300]                              | [1, 6616, 4]                                
2 | loss | MSELoss          | 0      | ?                                           | ?                                           
3 | mse  | MeanSquaredE

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.008
Epoch 0, global step 18: 'val_loss' reached 0.00828 (best 0.00828), saving model to '/mnt/e/Work/chi_phuong_co_su/log/epoch=0-val_loss=0.01.ckpt' as top 3


...

Epoch 79, global step 1440: 'val_loss' was not in top 3


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 80, global step 1458: 'val_loss' was not in top 3


Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 30 records. Best score: 0.002. Signaling Trainer to stop.
Epoch 81, global step 1476: 'val_loss' was not in top 3


In [16]:
training_module.validate_and_testing_lstm(data_module=flight_dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      val_loss_epoch       │   0.002143095014616847    │
│       val_mse_epoch       │   0.002143095014616847    │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_loss_epoch      │   0.002164205303415656    │
│      test_mse_epoch       │   0.002164205303415656    │
└───────────────────────────┴───────────────────────────┘

In [18]:
# Tensorboard
%tensorboard --logdir /mnt/e/Work/chi_phuong_co_su/log

Reusing TensorBoard on port 6007 (pid 236788), started 4:22:22 ago. (Use '!kill 236788' to kill it.)